**Problem Statment**

In this file Predict Reviews.ipynb we will be scrapping reviews from IMDB by giving an imdb id of movie and then we will predict which of the reviews are positive and negative by loading alrady built model.

# Step 1 - Importing the libraries

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import json
import bs4 as bs
import urllib.request
import pickle

# Step 2 - Loading already built model

In [4]:
# Load Model Config

with open('/content/drive/My Drive/Data Science Projects/dataset_movies/model_config.json') as json_file:
    json_config = json_file.read()
model = tf.keras.models.model_from_json(json_config)

# Load weights

model.load_weights('/content/drive/My Drive/Data Science Projects/dataset_movies/weights_only.h5')

with open('/content/drive/My Drive/Data Science Projects/dataset_movies/tokenizer.pkl', 'rb') as file:
  tokenizer = pickle.load(file)

with open('/content/drive/My Drive/Data Science Projects/dataset_movies/Tvalue.pkl', 'rb') as file:
  T = pickle.load(file)

# Step 3 - Web Scrapping to get user reviews for given movie id from IMDB

In [5]:
imdb_id = 'tt1409024'

# web scraping to get user reviews from IMDB site

sauce = urllib.request.urlopen('https://www.imdb.com/title/{}/reviews?ref_=tt_ov_rt'.format(imdb_id)).read()
soup = bs.BeautifulSoup(sauce,'lxml')
soup_result = soup.find_all("div",{"class":"text show-more__control"})

reviews_list = [] # list of reviews

for reviews in soup_result:  
  if reviews.string:
    reviews_list.append(reviews.string)      
        
movie_review_list = pd.Series(reviews_list)
movie_vector = tokenizer.texts_to_sequences(movie_review_list)
movie_vector = pad_sequences(movie_vector, maxlen = T)

# Step 4 - Predicting the reviews

In [6]:
yhat = model.predict(movie_vector)

result = []

for i in yhat:
  if(i>0.5):
    result.append('Good')
  else:
    result.append('Bad')

print(movie_vector)

[[   0    0    0 ...    5    1  210]
 [   0    0    0 ... 1411  655  353]
 [   0    0    0 ...    5   26  287]
 ...
 [   0    0    0 ...    4  619  214]
 [   0    0    0 ...   16    2  166]
 [   0    0    0 ...   25  106  176]]


In [12]:
reviews = pd.Series(data = reviews_list)
result = pd.Series(data = result)

final = pd.concat([reviews, result], axis =1)
print(final)

                                                   0     1
0  Men in Black 3 is fun, if flawed. Hugely bette...   Bad
1  It is really hard to believe that it has been ...  Good
2  This part of the Men in Black series is my fav...  Good
3  Interesting enough as time travel with aliens ...   Bad
4  Ending was good, beginning good. The hour and ...   Bad
5  Well it's been 10 years since the dismal MIB2 ...   Bad
6  Let's just get this out of the way: Josh Broli...   Bad
7  We loved parts I and II and it was enough. Rig...   Bad
8  Went to MIB III because it got a fresh ratings...   Bad


**Conclusion**

As seen we have predicted the reviews to be good or bad with already created NLP model.
This was the last file of complete movie recommendation project where we not only used web scrapping, data collection, EDA, data preprocessing, Cosine Similarity techniques but also used Deep Learning, NLP, API data collection, Sentiment Analysis, Model Deplyoment etc